# API Request Example to VRR
Define the name of the API endpoint you want to query and the parameters you need to pass.

In [78]:
# -*- coding: utf-8 -*-
# imports
import requests
import pandas as pd
import uuid
import time
from datetime import datetime


In [ ]:
# add api request function
# https://efa.vrr.de/standard/



# Build the request parameters as a dictionary

def full_api_request(datetime_dt, place_dm, name_dm):
    params = {
        "language": "de",
        "mode": "direct",
        "outputFormat": "JSON",
        "type_dm": "stop",
        "useProxFootSearch": 0,
        "useRealtime": 1,
        "itdDateDay": datetime_dt.day,
        "itdDateMonth": datetime_dt.month,
        "itdDateYear": datetime_dt.year,
        "itdTimeHour": datetime_dt.hour,
        "itdTimeMinute": datetime_dt.minute,
        "place_dm": place_dm,
        "name_dm": name_dm,  # Replace with the desired stop name
    }
    
    # Define the API URL
    API_URL = "https://efa.vrr.de/standard/XML_DM_REQUEST"

    # Send the request to the API
    response = requests.get(API_URL, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response with utf-8 encoding
        data = response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")

    def make_uid(stop, scheduled_time, line):
        base = f"{stop}|{scheduled_time}|{line}"
        return str(uuid.uuid5(uuid.NAMESPACE_DNS, base))

    # Extract stops, delay times, and scheduled departures from the 'data' variable
    departures = data.get('departureList', [])

    results = []
    for dep in departures:
        # Extract relevant information from each departure
        stop_name = dep.get('stopName')
        platform = dep.get('platformName', dep.get('platform'))
        scheduled = dep.get('dateTime', {})
        scheduled_time = f"{scheduled.get('hour', '')}:{str(scheduled.get('minute', '')).zfill(2)}"
        real = dep.get('realDateTime', {})
        real_time = f"{real.get('hour', '')}:{str(real.get('minute', '')).zfill(2)}" if real else None
        line = dep.get('servingLine', {}).get('number')
        direction = dep.get('servingLine', {}).get('direction')
        delay = dep.get('servingLine', {}).get('delay')
        cancelled = dep.get('servingLine', {}).get('cancelled')
        connection_exists = not (str(cancelled) == "1")

        # Create a unique identifier for the departure
        uid = make_uid(stop_name, scheduled_time, line)

        results.append({
            'uuid': uid,
            'stop': stop_name,
            'platform': platform,
            'line': line,
            'direction': direction,
            'scheduled_departure': scheduled_time,
            'real_departure': real_time,
            'delay_min': int(delay) if delay not in (None, '', '-9999') else None,
            'connection_exists': connection_exists
        })

    df_departures = pd.DataFrame(results)
    print(f"Found {len(df_departures)} departures.")

    # make sure utf-8 encoding is used - vomit on sight :)
    lut = {"Ã¼": "ü", "Ã¶": "ö", "Ã¤": "ä", "ÃŸ": "ß", "Ã": "ß"}
    for col in ['stop', 'direction']:
        df_departures[col] = df_departures[col].replace(lut, regex=True)
    return df_departures, response.status_code

In [ ]:
# Define the datetime for the request
datetime_dt = datetime.now()

# Define the place and name for the stop
place_dm = "Gelsenkirchen"
name_dm = "HBF"

placename_list = [("Duisburg", "HBF"), ("Mönchengladbach", "HBF"), ("Wuppertal", "HBF"), ("Bochum", "HBF"), ("Dortmund", "HBF"), ("Essen", "HBF"), ("Düsseldorf", "HBF")]
placename_list = [("Duisburg", "HBF")]

final_df = pd.DataFrame()
# Make the API request and get the DataFrame
for place_dm, name_dm in placename_list:
    print(f"Requesting data for {place_dm} - {name_dm}")
    df, status_code = full_api_request(datetime_dt, place_dm, name_dm)
    if not df.empty:
        final_df = pd.concat([final_df, df], ignore_index=True)

# Display the final DataFrame
final_df

Requesting data for Duisburg - HBF
Found 40 departures.


,uuid,stop,platform,line,direction,scheduled_departure,real_departure,delay_min,connection_exists
0,16592eeb-ce03-5f9a-ad2b-18b2e471e318,Duisburg Hbf,2,RE11 (RRX),Düsseldorf Hbf,16:07,None,NaN,True
1,5de4c9fd-ea74-50c0-81c7-f67ddaee53cc,Duisburg Hbf,1 B-D,RB33,"Aachen, Hbf",16:35,16:43,8.0,True
2,6bba863d-bcd7-5ac5-9a11-1cac608cbd83,Duisburg Hbf,2,RE2,Düsseldorf Hbf,16:36,16:45,9.0,True
3,9590a5af-5e0f-5a8c-ae5c-ed551315658a,Duisburg Hbf,3,901,Mülheim Hauptbahnhof,16:38,16:43,5.0,True
4,93c68386-ca6c-5297-853e-2d166e920117,Duisburg Hbf,2,903,Duisburg Rheintörchenstraße,16:40,16:44,4.0,True
5,0c931867-b827-50d5-b956-f40ef00c26e6,Duisburg Hbf,5,921,DU-Hbf Osteingang,16:40,16:43,3.0,True
6,d22b868d-eb76-502c-99bd-6b5bb2929b99,Duisburg Hbf,1,U79,Duisburg Meiderich Bf,16:40,16:42,2.0,True
7,9a4a515a-7a4c-56c0-8c6e-7aa4d4490bb9,Duisburg Hbf,3,901,Duisburg Zoo/Uni,16:43,None,NaN,True
8,d0ba636e-a2ee-539a-8357-5116d74a8027,Duisburg Hbf,5,934,Großenbaum Bf Ost,16:43,16:44,1.0,True
9,d4617880-bd56-54fa-8fdd-5014dd8226a7,Duisburg Hbf,12,ICE 1030 InterCityExpress,Hamburg Altona,16:43,16:45,2.0,True


In [89]:
delay_min = 0.4
placename_list = [("Duisburg", "HBF"), ("Mönchengladbach", "HBF"), ("Wuppertal", "HBF"), ("Bochum", "HBF"), ("Dortmund", "HBF"), ("Essen", "HBF"), ("Düsseldorf", "HBF")]

# determine total number of requests
total_requests = len(placename_list)
# calculate delay in milliseconds
delay_s = delay_min * 60  # convert minutes to milliseconds
# time the actual requests so that they space out over the delay time
request_delay = delay_s / total_requests

# initialize csv for final_df only if it does not exist
import os
if os.path.exists('final_departures.csv'):
    print("CSV file already exists, loading existing data...")
    df = pd.read_csv('final_departures.csv')
else:
    df = pd.DataFrame(columns=[
        'uuid', 'stop', 'platform', 'line', 'direction',
        'scheduled_departure', 'real_departure', 'delay_min', 'connection_exists'
    ])
    df.to_csv('final_departures.csv', index=False)
    print("Initialized CSV file")

# Main loop
print(f"Total requests: {total_requests}, Delay per request: {round(request_delay/60, 2)} minutes.")
print("Starting the request loop...")

while True:
    print("Starting a new cycle of requests...")
    datetime_dt = datetime.now()
    # load csv file to run_df
    run_df = pd.read_csv('final_departures.csv')

    for place_dm, name_dm in placename_list:
        print(f"Requesting data for {place_dm} - {name_dm}")
        df, status_code = full_api_request(datetime_dt, place_dm, name_dm)
        # merge the DataFrame with run_df and ignore UUID duplicates
        if not df.empty:
            run_df = pd.concat([df for df in [run_df, df] if df.dropna(how='all').shape[0] > 0], ignore_index=True).drop_duplicates(subset='uuid', keep='last')
            print(f"Added {len(df)} new departures to the DataFrame. Sleeping for {round(request_delay/60, 2)} minutes.")
        else:
            print(f"No new departures found for {place_dm} - {name_dm}. Sleeping for {round(request_delay/60, 2)} minutes.")

        run_df.to_csv('final_departures.csv', index=False)
        print(f"Updated CSV file with {len(run_df)} total departures.")
        time.sleep(request_delay)  # Delay to space out requests

    print("Next cycle...")

CSV file already exists, loading existing data...
Total requests: 7, Delay per request: 0.06 minutes.
Starting the request loop...
Starting a new cycle of requests...
Requesting data for Duisburg - HBF
Found 40 departures.
Added 40 new departures to the DataFrame. Sleeping for 0.06 minutes.
Updated CSV file with 318 total departures.
Requesting data for Mönchengladbach - HBF
Found 40 departures.
Added 40 new departures to the DataFrame. Sleeping for 0.06 minutes.
Updated CSV file with 357 total departures.
Requesting data for Wuppertal - HBF
Found 40 departures.
Added 40 new departures to the DataFrame. Sleeping for 0.06 minutes.
Updated CSV file with 396 total departures.
Requesting data for Bochum - HBF
Found 40 departures.
Added 40 new departures to the DataFrame. Sleeping for 0.06 minutes.
Updated CSV file with 436 total departures.
Requesting data for Dortmund - HBF
Found 40 departures.
Added 40 new departures to the DataFrame. Sleeping for 0.06 minutes.
Updated CSV file with 476 

KeyboardInterrupt: 